<a href="https://colab.research.google.com/github/ngoan22mse23088/python4engineers-assignments/blob/master/Assignments_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# cài đặt packages version như sau
!pip install opendatasets
!pip install mysql-connector-python
!pip install pandas matplotlib
!apt-get update install mysql-server mysql-client -y
!service mysql start

# cài đặt môi trường mysql trên colab với thông số :
# host : localhost
# user : root
# pass : root
# database : laptop
!mysql -u root -p -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root';"
!mysql -u root -p -e "CREATE DATABASE laptop;"
!mysql -u root -p -e "SHOW DATABASES;"
!mysql -u root -p -e "GRANT ALL PRIVILEGES ON laptop.* TO 'root'@'localhost';"

# cài đặt thư viện tiền xử lý
# Thêm các thư viện cơ bản để xử lý
import re
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

# Dùng thư viện BeautifulSpup để scraping dữ liệu về
url = 'https://gearvn.com/collections/laptop-gaming-ban-chay'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
products = soup.find_all('div', {'class': 'product-row'})
data = []

# Đọc từng thẻ trong file html để lấy data
for product in products:
    # Đọc giá trị tên
    name = product.find('h2', {'class': 'product-row-name'}).text.strip()
    # Đọc giá trị cũ
    old_price = product.find(
        'div', {'class': 'product-row-price'}).find('del').text.strip()
    # Đọc giá mới
    new_price = product.find('span', {'class': 'product-row-sale'}).text.strip()
    # Đọc giá trị giảm giá
    percent_discount = product.find(
        'div', {'class': 'new-product-percent'}).text.strip()
    # Đọc giá trị best saller (trả về 1 nếu là best seller và ngược lại là 0)
    best_seller = 1 if product.find(
        'span', {'class': 'ico-product ico-km'}) else 0
    # Phân tích chuỗi để lấy tên và brand
    # Case Laptop Gaming và Laptop gaming và Laptop
    match = re.search(r'^laptop gaming (.*)', name.lower())
    if match:
        product_name = match.group(1).strip()
        brand_match = re.search(r'^(.*?)\s+(.*)', product_name)
        if brand_match:
            brand = brand_match.group(1).strip()
            product_name = brand_match.group(2).strip()

    # Lấy giá củ (tính đến đơn vị triệu đồng)
    old_price = int(re.sub(r'[^\d]+', '', old_price))/1000000
    # Lấy giá mới (tính đến đơn vị triệu đồng)
    new_price = int(re.sub(r'[^\d]+', '', new_price))/1000000
    # Lấy phần trăm giảm giá
    percent_discount = int(re.sub(r'[^\d]+', '', percent_discount))

    # Đưa vào mảng để trực quan hóa dữ liệu
    data.append({
        'Name': product_name,
        'Brand': brand,
        'OldPrice': old_price,
        'NewPrice': new_price,
        'PercentDiscount': percent_discount,
        'BestSeller': best_seller
    })

df = pd.DataFrame(data)
# In laptop đã phân tích
print(df)

# Trực quan trung bình và trung vị giá bán cũ và mới với các Brand.
agg_df = df.groupby("Brand").agg({"OldPrice": ["mean", "median"], "NewPrice": ["mean", "median"]})
agg_df.columns = ["Giá cũ (trung bình)", "Giá cũ (trung vị)", "Giá mới (trung bình)", "Giá mới (trung vị)"]
agg_df = agg_df.reset_index()

# Vẽ biểu đồ boxplot
fig, axs = plt.subplots(ncols=2, figsize=(12, 6))

sns.boxplot(x="Brand", y="OldPrice", data=df, ax=axs[0])
sns.boxplot(x="Brand", y="NewPrice", data=df, ax=axs[1])

axs[0].set_title("Giá cũ")
axs[1].set_title("Giá mới")

axs[0].set_ylabel("Giá (triệu đồng)")
axs[1].set_ylabel("Giá (triệu đồng)")
plt.show()

# Nhận xét
# Các Brand Laptop Gaming trong dataset có giá bán cũ và mới từ 10 triệu đồng đến 30 triệu đồng, với giá bán mới trung bình và trung vị thấp hơn giá bán cũ.
# Brand MSI có giá bán cũ và mới cao hơn so với các Brand khác, với giá bán mới trung bình và trung vị đều trên 20 triệu đồng.
# Brand Asus có giá bán mới trung bình thấp hơn so với các Brand khác, trong khi Brand Acer có giá bán mới trung vị thấp hơn.

# Trực quan đếm số sản phẩm của các Brand
df = pd.DataFrame(data)
sns.set(style="darkgrid")
plt.figure(figsize=(12, 6))

# Vẽ biểu đồ
ax = sns.countplot(x="Brand", data=df, order=df['Brand'].value_counts().index)
ax.set_title("Số lượng sản phẩm của từng Brand")
ax.set_xlabel("Brand")
ax.set_ylabel("Số lượng sản phẩm (cái)")
plt.show()

# Tính giá trị giảm giá cao nhất và thấp nhất của từng Brand
discount_df = df.groupby("Brand").agg({"PercentDiscount": ["max", "min"]})
discount_df.columns = ["MaxDiscount", "MinDiscount"]
discount_df = discount_df.reset_index()

# Vẽ biểu đồ
fig, ax = plt.subplots(figsize=(10, 6))
x = discount_df['Brand']
y1 = discount_df['MaxDiscount']
y2 = discount_df['MinDiscount']

# Vẽ biểu đồ đường thẳng
ax.plot(x, y1, color='red', marker='o', label='Giảm giá cao nhất')
ax.plot(x, y2, color='blue', marker='o', label='Giảm giá thấp nhất')

# Đặt tiêu đề và tên trục
ax.set_title('Giá trị giảm giá cao nhất và thấp nhất của từng Brand')
ax.set_xlabel('Brand')
ax.set_ylabel('Giảm giá (%)')

# Hiển thị chú thích màu
ax.legend(loc='best')
plt.show()

# Nhận xét: Boxplot cho phép chúng ta so sánh mức giảm giá và giá mới của các sản phẩm của các brand khác nhau.
# Chúng ta có thể quan sát được rằng, các sản phẩm của brand Dell, MSI và Lenovo thường có mức giảm giá cao hơn so với các brand khác.
# Tuy nhiên, các sản phẩm của brand Acer và Asus lại có giá trị trung bình và median cao hơn so với các brand khác.
# Boxplot cũng cho thấy rằng, các brand khác nhau có sự phân tán khác nhau về giá và mức giảm giá của sản phẩm của mình.

# Trực quan mức giá mới của các sản phẩm là BestSeller
# create empty lists to store data
best_sellers = []
new_prices = []
old_prices = []

# Lấy sản phẩm là best seller
for item in data:
    if item['BestSeller'] == 1:
        best_sellers.append(item['Name'])
        new_prices.append(item['NewPrice'])
        old_prices.append(item['OldPrice'])

# Tạo bar chart
fig, ax = plt.subplots(figsize=(10, 6))
plt.bar(best_sellers, new_prices)
plt.xticks(rotation=90)
plt.ylabel('Giá mới (triệu đồng)')
plt.title('Giá mới của sản phẩm Best Seller')

# Tạo line chart
plt.plot(best_sellers, old_prices, color='red', linewidth=2.5, label='Giá cũ')
plt.legend()

# Vẽ biểu đồ
plt.show()

# Nhận xét: 
# Dựa trên biểu đồ, ta có thể thấy mức giá mới của các sản phẩm BestSeller khác nhau. 
# Sản phẩm có giá mới cao nhất là tầm 28 đến 30 đơn vị và sản phẩm có giá mới thấp nhất là khoảng 16 đơn vị. 
# Các sản phẩm khác có mức giá mới nằm trong khoảng này. 
# Ta có thể sử dụng thông tin này để so sánh giá của các sản phẩm BestSeller và đưa ra quyết định mua sắm phù hợp.